### Using CSV

In [1]:
import numpy as np
import pandas as pd

import sklearn
from sklearn.decomposition import TruncatedSVD

> Check why the csv functionality isn't working. 

In [2]:
columns = ['user_id', 'item_id', 'item_name', 'Rating']
frame = pd.read_csv('cdata.csv',  names=columns, sep='\s+')
frame.head()

,,,,user_id,item_id,item_name,Rating
"506246,104000100187,BENU",odměna,-,PÉČE,O,ZUBY,15%,"sleva,49.26"
"506246,104000100150,BENU",odměna,-,Muconasal,"Plus,17.89",NaN,NaN,NaN
"506246,104000100168,BENU",odměna,-,OCUTEIN,Da,Vinci,"Academia,50.24",NaN
"506246,104000100220,BENU",odměna,-,SEZÓNA,"LÉTO,63.45",NaN,NaN,NaN
"506246,104000100057,BENU",odměna,-,"Coldrex,34.22",NaN,NaN,NaN,NaN


### Using Network connection

In [3]:
# Database Connection
import psycopg2
import pandas as pd
df = pd.DataFrame()
output = []
dbname = "datasapienspragueredshift001"
port = 5439
user = "datasapiens"
password = "Datasapiens01"
host = "datasapiens-prague-redshift-001-dense-storage.ckg2sdykjtd6.us-east-1.redshift.amazonaws.com"
con = psycopg2.connect(dbname = dbname, host = host, port = port, user = user, password= password)
cur = con.cursor()

# Query input
cur.execute("SELECT customer_lvl2_id_1, coupon_product_lvl4_code_1, coupon_name, composite_not_norm_score FROM datasapiens_prague_benu.temp_am_final_score_table ORDER BY customer_lvl2_id_1 LIMIT 1000000;")
rows = cur.fetchall()
print("Received as rows")

Received as rows


In [4]:
# Making a DataFrame
columns = ['user_id', 'item_id', 'item_name', 'Rating']
df = pd.DataFrame.from_records(rows, columns = columns)
df.head()

,user_id,item_id,item_name,Rating
0,506246,104000100187,BENU odměna - PÉČE O ZUBY 15% sleva,49.2641484708555
1,506246,104000100150,BENU odměna - Muconasal Plus,17.8935396437190
2,506246,104000100168,BENU odměna - OCUTEIN Da Vinci Academia,50.2377314877980
3,506246,104000100220,BENU odměna - SEZÓNA LÉTO,63.4493992755955
4,506246,104000100057,BENU odměna - Coldrex,34.2231676897300


In [5]:
# Number of products (coupons)
df.groupby('item_id')['Rating'].count().sort_values(ascending=False).head()

item_id
104000100266    53681
104000100181    41336
104000100029    29793
104000100030    29793
104000100187    27561
Name: Rating, dtype: int64

In [6]:
# To find the maximum used product (coupon)
filter = df['item_id'] == 104000100266 
df[filter]['item_name'].unique()

array([], dtype=object)

In [7]:
df.describe()

,user_id
count,1000000.000000
mean,679327.661655
std,105578.662588
min,506246.000000
25%,584311.000000
50%,676753.000000
75%,769753.000000
max,876855.000000


> Converting Data types

In [8]:
df.dtypes

user_id       int64
item_id      object
item_name    object
Rating       object
dtype: object

In [9]:
df = df.convert_objects(convert_numeric=True)

C:\Users\Aman Deep Middha\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


In [10]:
df.dtypes

user_id        int64
item_id      float64
item_name     object
Rating       float64
dtype: object

In [11]:
# Building a Utility matrix
rating_crosstab = df.pivot_table(values='Rating', index='user_id', columns='item_name', fill_value=0)
rating_crosstab.head()

MemoryError: 

#### Transposing the matrix

In [ ]:
rating_crosstab.shape

In [ ]:
X = rating_crosstab.T
X.shape

In [ ]:
SVD = TruncatedSVD(n_components=15, random_state=17)

resultant_matrix = SVD.fit_transform(X)

resultant_matrix.shape

In [ ]:
corr_mat = np.corrcoef(resultant_matrix)
corr_mat.shape

In [ ]:
item_names = rating_crosstab.columns
item_list = list(item_names)

vitamin = item_list.index('BENU odm\xc4\x9bna - VITAMINY 15% sleva')
vitamin

In [ ]:
corr_item = corr_mat[241]
corr_item.shape

In [ ]:
list(item_names[(corr_item<1.0) & (corr_item > 0.90)])